In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>

# <h1><center><ins>FLASK</ins></center></h1>
<h1><center>Práctica guiada:</center></h1>
<img src="img/01_flask.jpg" alt="Drawing" style="width: 400px;"/>

<a id="tabla_contenidos"></a> 
## Tabla de Contenidos

### <a href='#0'>0. Objetivos de la Notebook</a>


### <a href='#1'>1. Introducción</a>
- #### <a href='#1.1'>1.1 Simular nuestro propio "SERVIDOR"</a>
- #### <a href='#1.2'>1.2 Flask</a>


### <a href='#2.'>2. Simulamos el armado de nuestra propia API</a>

***

<a id="0"></a> 
## 0. Objetivos de la Notebook

<div id="caja11" style="float:left;width: 100%;">
  <div style="float:left;width: 9%;"><img src="../../../common/icons/haciendo_foco.png" style="align:left"/> </div>
  <br>
  <div style="float:left;width: 85%;">
      <label>Levantar los modelos ya entrenados usando </label>
      <a class="reference internal" href=https://docs.python.org/3/library/pickle.html><b><code>Pickle</code></b></a> 
  <div style="float:left;width: 85%;">
      <label>Usar <a class="reference internal" href=https://flask.palletsprojects.com/en/1.1.x/><b><code>Flask</code></b></a> para simular que armamos una API en un servidor y hacerle diferentes pedidos.

<a href='#tabla_contenidos'>Volver a TOC</a>

---

<a id="1"></a> 
## 1. Introducción

<a id="1.1"></a> 
### 1.1 Simular nuestro propio "SERVIDOR"
<img src="img/02_flask.jpg" alt="Drawing" style="width: 350px;"/>

<br>  

El `Deploy` un modelo de machine-learning significa integrar el modelo en un ambiente de producción, en el cual puede tomar un input y retornar un output:

<br>  

<img src="img/03_flask.jpg" alt="Drawing" style="width: 800px;"/>

<br>  


Ahora vamos a simular este contextos de producción usando los modelos entrenados que guardamos en la notebook anterior. Vamos a llevar esos archivos a nuestro "SERVIDOR" y lo que vamos a hacer es armar nuestra API para que otros servicios o usuarios de afuera (según como esté armada la infraestructura en nuestro trabajo) puedan acceder a los resultados de nuestros modelos.

**Algo a tener en cuenta:** es importante que en nuestro servior tengamos todos los requerimientos, librerías y archivos (por ejemplo, los modelos, o databse si vamos a entrenar un modelo) para nuestra API (al final les dejamos varios links que explican cómo hacer esto para quien tenga curiosidad y ganas de ver cómo se hace). 



<a id="1.2"></a> 
### 1.2 Flask
<img src="img/04_flask.jpg" alt="Drawing" style="width: 500px;"/>

<br>  


<a class="reference internal" href=https://flask.palletsprojects.com/en/1.1.x/><b><code>Flask</code></b></a> es un conjunto de herramientas para desarrollar aplicaciones web que también es definido (por la misma librería) como un web micro-framework. "Micro" en el término microframework se refiere a que Flask tiene un nucleo simple pero que permite agregar extensiones con mucha faciliad. Es decir, que no obliga a usar un determinado tipo de database, y permite cambiar fácilmente el motor que se usa como template para mostrar la aplicación web. Además, la librería está escrita en lenguaje Python y hecha para aplicaciones del mismo lenguaje. 



<br>  

`Flask` permite llevar nuestros modelos de machine-learning a un entorno e aplicación web de una manera sencilla, y está basada en dos componentes principales: 
        - **Web Server Gateway Interface (WSGI):** es un conjunto de convenciones y especificaciones que describen cómo una servidor web se comunica con otras aplicaciones web, y cómo las aplicaciones web procesan un pedido en lenguaje Python (si quieren saber un poco más: https://en.wikipedia.org/wiki/Web_Server_Gateway_Interface). 
        - **Jinja2**:  es un motor de templates para Python que se encuentra configurado automáticamente en el framework Flask y permite renderizar páginas web (si quieren más información: https://pypi.org/project/Jinja2/). Es decir,  permite definir cómo se va a mostrar la información que devuelva nuestra API en la web (y acá pueden encontrar algunos templates HTML https://juncotic.com/jinja2-en-flask-introduccion/).
        

#### Manos a la obra: 
Hagamos una primera prueba de "juguete" para analizar cómo podemos simular un servidor local (y de paso probar un template). Vamos a empezar con algo muy simple, un servidor que nos devuelva la frase "Hola, mundo!" con una foto del mundo. 


<br>  

<img src="img/05_flask.jpg" alt="Drawing" style="width: 300px;"/>

### Imports:

In [ ]:
# importamos las librerías que vamos a usar
from flask import  Flask, request, jsonify, render_template, send_file
import pandas as pd
import numpy as np
import json
import pickle

Analicemos nuestro ejemplo base y sus distintas partes:

In [ ]:
# En una variable instanciamos la clase Flask con el nombre de nuestra api (en este caso la llamamos "hola").
# Además, le pasamos un argumento "template_folder" donde le indicamos en qué carpeta se encuentra la plantilla
# HTML que vamos a utilizar para formatear el output de nuestra API. Si van a la carpeta plantilla, van a encontrar
# un archivo "template1.html" que está basado en un template de esta página: 
# https://juncotic.com/jinja2-en-flask-introduccion/ . Este es el template que Flask en base a Jinja2 va a renderizar.
app = Flask('bienvenida',template_folder='plantillas')

In [ ]:
# "@app.route" es un decorador que modifica la función que se define luego ("holas"). Este decorador convierte al 
# string que está entre paréntesis ('/hola') en la ruta o endpoint con el cual nuestra API va a encontrar esta función.
@app.route('/hola')
def holas():
    texto = "¡Hola, mundo!"
    return render_template('template1.html' , texto = texto)
    # en el return de la función, indicamos que devuelva la variable texto pero usando como render el archivo HTML
    # que tenemos en la carpeta plantilla. 
    
@app.route('/rock')
def get_image():
    filename = 'plantillas/rock.png'
    return send_file(filename, mimetype='image/png')    
    

In [ ]:
# Cuando ejecuten esta celda, verán que la notebook se queda "trabajando" sin terminar nunca. Eso es porque lo que
# esta línea hace que la notebook simule ser un servidor que está poniendo a disposición a nuestra api "bienvendia"
# para que podemos acceder localmente. Lo que tienen que hacer es lo siguiente: mientras se está ejecutando esta celda
# ejecutar esta celda antes de entrar en la dirección): http://localhost:5001/hola
app.run(host='0.0.0.0', port = 5001)

http://localhost:5001/hola

Si lo anterior funcionó correctamente, les tiene que haber aparecido en su navegador algo como esto: 

<br>  

<img src="img/06_flask.jpg" alt="Drawing" style="width: 500px;"/>


<br>  

Fijense que el kernel de su notebook sigue funcionando y no van a poder ejecutar líneas de código en otras celdas. Para `volver a hacer uso de la notebook (y que deje de actuar como nuestro servidor) tienen que ir a la celda anterior (la que tiene la línea "app.run(host='0.0.0.0')" y darle detener`, así dejamos de disponibilizar localmente en el puerto 500 la función de nuestra API "bienvenida").

Con esto ya estamos listos para armar una API más compleja en la cual utilicemos los modelos que entrenamos en la otra notebook. 

<a href='#tabla_contenidos'>Volver a TOC</a>

***

<a id="2."></a> 
## 2. Simulamos el armado de nuestra propia API

### Definimos el nombre de nuestra API y una primera ruta con las instrucciones de su uso

Levantamos la lista de variables dummmies para pasarlas en las instrucciones

In [ ]:
with open('dummies_order.pkl', 'rb') as f_dummy:
    dummies_encoder = pickle.load(f_dummy)

dummies_encoder

Instanciamos y definimos nuestra función de bienvenida

In [ ]:
app = Flask('Predictor de examenes')

In [ ]:
@app.route("/")
def hello():
    instrucciones_html = '''Para utilizar esta api se debe poner 
    el nombre del modelo y los predictores %s''' % dummies_encoder
    
    return instrucciones_html

In [ ]:
# Tal como hicimos antes, ejecutamos esta celda e ingresamos a la siguiente dirección para ver su resultado 
# (fijense que esta vez no estamos usando la función "render_template" así que el output será un string de 
# texto sin formato): http://localhost:5002/
app.run(host='0.0.0.0', port = 5002)

http://localhost:5002/

Si todo funcionó correctamente, tienen que estar viendo lo que nuestra API devuelve cuando el endpoint no contiene ninguna especificación:

<br>  

<img src="img/07_flask.jpg" alt="Drawing" style="width: 800px;"/>


<br>  

Recuerden ahora para continuar, detener la ejecución de la línea `app.run(host='0.0.0.0')` de la celda previa.

### Probamos el método GET
<br>  

<img src="img/10_flask.jpg" alt="Drawing" style="width: 400px;"/>


<br>  
En la clase de APIs vimos que `HTTP` es uno de los protocolos base para la transferencia de datos en la web y para los protocolos de intercambio entre clientes y servidores. Este protocolo está basado en la architectura `REST` (Representational State Transfer) que determina una serie de reglas para el diseño de un sistema distribuido en una red (como internet). Esta arquitectura plantea una serie de reglas que los desarrolladores deben seguir al momento de crear las APIs, y a las que adhiere Flask. 

En el contexto de la arquitectura `REST` nos encontramos con la siguiente estructura básica: 
<br>  

<img src="img/08_flask.jpg" alt="Drawing" style="width: 800px;"/>


<br>  



Como ya vimos en la clase de APIs existen distintos verbos HTTP que regula la relación entre cliente y servidor (de los cuales nosotros veremos cómo definir dos de ellos en nuetra API): 


<br>  


- **GET:** sólo se usa para obtener información que brinda el servidor. Un pedido de esta clase no debería tener un efecto en la información en el servidor. 

- **POST:** este pedido permite enviar datos al servidor usando información HTML (puede generar un cambio). 

- **DELETE:** borra la representación de un recurso. 

- **PUT:** reemplaza la representacióon con los datos en la petición. 


<br>  

Nuestra aplicación va a saber identificar qué solicitud se le está haciendo a partir de la `URL (Uniform Resource Locator)` que es la ruta que indica dónde se puede encontrar un recurso: 
<br>  

<img src="img/11_flask.jpg" alt="Drawing" style="width: 800px;"/>


<br>  


Fijensé que en el ejemplo que hicimos antes también definimos una URL (aunque local) en la que el `puerto` que definimos era el **500**, el `resource path` era la **/** (o su ausencia), y no teníamos ninguna `query` que determinara lo que se iba a devolver a partir de la solicitud. 

<br>  

Ahora vamos a definir otro endpoint en nuestra API, donde vamos a explicitar que el método de este endpoint es GET y vamos a probar cómo podemos pasarle diferentes query al endpoint y cómo las recibe el servidor:



In [ ]:
# Definimos un nuevo endoint o resource path (prueba_get) y explicitamos el método.
@app.route("/prueba_get",methods=['GET'])
def pruebaGet():
    #request.args
    
    return jsonify(request.args)
    # request.args nos va a permitir capturar la información que pasemos en las query como si fueran
    # diccionarios (ya veremos ejemplos más concretos más adelante), en este caso no definimos 
    # ninguna que la función pruebaGet necesite ya que lo estamos haciendo para probar su funcionamiento. 
    # Por otro lado, el método jsonify pasa a json los valores que le pasemos a la solicitud para que 
    # puedan ser devueltos por la API en formato json. 

In [ ]:
# disponibilizamos el servidor, y probamos el siguiente llamado con dos query inventadas para ver qué nos devuelve: 
# http://localhost:5003/prueba_get?id=1555&math=10
# Fijensé que para unir dos query se usa el símbolo "&"
app.run(host='0.0.0.0', port = 5003)

http://localhost:5003/prueba_get?id=1555&math=10

Con la llamada anterior, les tendría que haber aparecido algo así: 
<br>  

<img src="img/12_flask.jpg" alt="Drawing" style="width: 600px;"/>


<br>  



Fijensé que tal como habíamos planteado, `request.args` toma las query y las transforma en un diccionario. Entender este funcionamiento es muy importante porque nos va permitir que nuestra API devuelva información específica o realice determinados procesos en función de cuál sea la información que se pase en las query. 

Ahora vamos a construir un recurso más complicado: vamos a armar una función que si le pasamos el nombre del modelo en la query, devuelva estadísticos de los puntajes de los alumnos en cada prueba de ese modelo. Además, le vamos a poder especificar si queremos que nos devuelva una versión simple de la estadística o una versión más completa (recuerden de detener la celda donde disponibilizamos nuestra API para poder seguir ejecutando las siguientes celdas).

In [ ]:
@app.route("/descriptivos_base",methods=['GET'])
def estadistica_Base():
    
    # 1) Importamos la base desde donde la tengamos almacenada en el "servidor"
    df=pd.read_csv('../Data/StudentsPerformance.csv')
    
    
    # 2) Sacamos los datos de las queries (chequeamos que estén y sino mandamos mensaje de error)
    if 'modelo' not in request.args.keys():
        return ('''Debés especificar en el pedido el modelo
                   que puede ser a) math, b)read or c)write ''')
    elif 'modelo' in request.args.keys():
        modelo=request.args['modelo']
    
    if 'metodo' not in request.args.keys():
        return ('''Debés especificar en el pedido el metodo. Puede ser 
                       a) metodo=simple que devuelve la media
                       b) metodo=completo que devuelve la media, el desvío y la mediana''')
    elif 'metodo' in request.args.keys():
        metodo=request.args['metodo']
    
    
    # 3) Elegimos el modelo del que queremos devolver información
    # Matematica
    if modelo=='math':
        if metodo=='simple':
            return jsonify({'media':df['math score'].mean()})
        elif metodo=='completo':
            return jsonify({'media':df['math score'].mean(),
                            'std':df['math score'].mean(),
                            'mediana':df['math score'].median()})
        else:
            return ('''Solo puedes hacer pedidos por los siguientes
                       metodos: 
                       a) metodo=simple que devuelve la media
                       b) metodo=completo que devuelve la media, el desvío y la mediana''')
    
    # Lectura
    if modelo=='read':
        if metodo=='simple':
            return jsonify({'media':df['reading score'].mean()})
        elif metodo=='completo':
            return jsonify({'media':df['reading score'].mean(),
                            'std':df['reading score'].mean(),
                            'mediana':df['reading score'].median()})
        else:
            return ('''Solo puedes hacer pedidos por los siguientes
                       metodos: 
                       a) metodo=simple que devuelve la media
                       b) metodo=completo que devuelve la media, el desvío y la mediana''')
        
    # Escritura
    if modelo=='write':
        if metodo=='simple':
            return jsonify({'media':df['writing score'].mean()})
        elif metodo=='completo':
            return jsonify({'media':df['writing score'].mean(),
                            'std':df['writing score'].mean(),
                            'mediana':df['writing score'].median()})
        else:
            return ('''Solo puedes hacer pedidos por los siguientes
                       metodos: 
                       a) metodo=simple que devuelve la media
                       b) metodo=completo que devuelve la media, el desvío y la mediana''')

In [ ]:
# Disponibilizamos nuestra API. Pueden probar pasar el siguiente llamado: 
# http://localhost:5004/descriptivos_base?modelo=math&metodo=simple
# y prueben qué sucede si pasan el siguiente: 
# http://localhost:5004/descriptivos_base?modelo=math&metodo=completo
# y prueben qué sucede si le pasan una query equivocada.
app.run(host='0.0.0.0', port = 5004)

http://localhost:5004/descriptivos_base?modelo=math&metodo=simple

http://localhost:5004/descriptivos_base?modelo=math&metodo=completo

Este es un ejemplo de lo que tendrían que haber obtenido con el segundo llamado: 
<br>  

<img src="img/13_flask.jpg" alt="Drawing" style="width: 600px;"/>



### Probamos ahora distintas formas para generar una predicción con nuestros modelos
<br>  

<img src="img/14_flask.jpg" alt="Drawing" style="width: 600px;"/>

<br> 

**CONVERTERS**

Vamos a armar ahora la función para hacer el pedido de la predicción. Las variables de nuestro modelo son dummies que pueden tomar valores de 0 y 1. Una primera forma de pasar los valores de un nuevo caso es a través de **converters** que nos permiten especificar partes del URL como variables de Python y pasárselas a la función asignada en la llamada. En esta página pueden encontrar información de cómo definir los distintos tipos de **converters**.

In [ ]:
with open('dummies_order.pkl', 'rb') as f_dummy:
    dummies_encoder = pickle.load(f_dummy)

dummies_encoder

In [ ]:
# definimos los converters asociados a las variables dummies y establecemos que son de tipo "int"
parametros_url = '<int:' + '>/<int:'.join(dummies_encoder) + '>'
parametros_url

In [ ]:
# pasamos en la URL asociada a la función los converters que definimos arriba y, además uno llamado <model> que va a
# tomar un string y con el cual vamos a poder definir cuál modelo de los tres que guardamos en la notebook anterior
# queremos ejecutar.
@app.route('/prediccion/<model>/' + parametros_url)
def prediccion(model, 
               gender_female, gender_male, 
               race_ethnicity_group_A, race_ethnicity_group_B, 
               race_ethnicity_group_C, race_ethnicity_group_D, race_ethnicity_group_E, 
               parental_level_of_education_associates_degree, 
               parental_level_of_education_bachelors_degree, 
               parental_level_of_education_high_school, parental_level_of_education_masters_degree, 
               parental_level_of_education_some_college, parental_level_of_education_some_high_school, 
               lunch_free_reduced, lunch_standard, 
               test_preparation_course_completed, test_preparation_course_none):
               # Fijense que en la función, tenemos que pasar como argumentos cada uno de los converters 
               # que definimos en la URL. 
    
    
    # 1) Levantamos el modelo seleccionado desde nuestro servidor local
    if model == 'math':
        
        # Cargamos el modelos desde nuestra carpeta local en el servidor usando Pickle
        with open('./math_model.pkl', 'rb') as f_math:
            modelo_matematicas = pickle.load(f_math)
        
        m = modelo_matematicas
        
    else:
        return 'modelo %s no existe' % model
    
    
    pred = m.predict([[gender_female, gender_male, 
               race_ethnicity_group_A, race_ethnicity_group_B, race_ethnicity_group_C, race_ethnicity_group_D, race_ethnicity_group_E, 
               parental_level_of_education_associates_degree, parental_level_of_education_bachelors_degree, parental_level_of_education_high_school, parental_level_of_education_masters_degree, parental_level_of_education_some_college, parental_level_of_education_some_high_school, 
               lunch_free_reduced, lunch_standard, 
               test_preparation_course_completed, test_preparation_course_none]])
    
    pred = 'La prediccion para matematicas es ' + str(pred)
    
    return pred

In [ ]:
# Ponemos a correr la api y le pasamos una URL. Dado que le estamos pasando la información con los converters
# le tenemos que pasar los valores de las dummies en la posición que corresponda: 
# http://localhost:5005/prediccion/math/1/0/1/0/0/0/0/1/1/0/0/0/1/0/1/0/1
app.run(host='0.0.0.0', port = 5005)

http://localhost:5005/prediccion/math/1/0/1/0/0/0/0/1/1/0/0/0/1/0/1/0/1

**QUERY**

Vamos a probar de pasarlo de otra manera, usando query.

In [ ]:
@app.route("/prediccion_get", methods=['GET'])
def prediccion_get():
    
    # 1) Tomamos los datos de las queries
    #    En este caso no definimos en la ruta ni en la función cuáles son los valores que va a tomar, sino que
    #    definimos el método GET y usamos "request.args" para traducir en diccionarios las query que pasemos. 
    #    Al convertirse en diccionarios, podemos usar las claves para guardar la información de cada query en 
    #    una variable, y luego operar con ellas como hacemos normalmente. 
    modelo=request.args['modelo']
    features=request.args['features']
    
    # 2) Levantamos el modelo seleccionado desde nuestro servidor local
    if modelo == 'math':
        
        # Cargamos el modelos desde nuestra carpeta local en el servidor usando Pickle
        with open('./math_model.pkl', 'rb') as f_math:
            modelo_matematicas = pickle.load(f_math)
        
        m = modelo_matematicas
        
    else:
        return 'modelo %s no existe' % model
    
    # 3) Hacemos la predicción
    # Es importante tener en cuenta que las query que levantamos con "request.args" vienen en formato json. 
    # Si las vamos a usar como texto, no tenemos que hacer nada, pero si son listas, diccionarios, int, floats
    # o booleanos, usamos "json.loads" para pasarlo al formato de objeto que querramos usar. En este caso, 
    # transformamos el texto "[1,0,1,0,0,0,0,1,1,0,0,0,1,0,1,0,1]" en una lista para poder convertirla en un 
    # dataframe y realizar la predicción. 
    datos = pd.DataFrame(json.loads(features)).T
    res = m.predict(datos)
    
    pred = 'La prediccion para matematicas es ' + str(res)
    
    return pred

In [ ]:
# Pasamos la nueva forma de hacer el llamado: 
# http://localhost:5006/prediccion_get?modelo=math&features=[1,0,1,0,0,0,0,1,1,0,0,0,1,0,1,0,1]
app.run(host='0.0.0.0', port = 5006)

http://localhost:5006/prediccion_get?modelo=math&features=[1,0,1,0,0,0,0,1,1,0,0,0,1,0,1,0,1]

**USANDO UN JSON + MÉTODO POST**

Ahora, vamos a probar enviar realmente datos al servidor con el método post usando un json y que este nos devuelva la solicitud. En los ejemplos anteriores también estábamos enviando información pero lo hacíamos usando partes de la URL, donde técnicamente no estábamos enviando información al servidor, sino indicando una ruta o camino de acceso. Ahora sí vamos a enviar esa misma información pero como un dato que va a ir al servidor. Por eso vamos a usar el método `POST` y no `GET`. 

In [ ]:
# definimos el método post en nuestra URL
@app.route('/api',methods=['POST'])
def predict_post():
    # obtengo los datos del request post.
    data = request.get_json(force=True)
    # data va a ser el json que le vamos a pasar:
    #https://kite.com/python/docs/flask.request.get_json
    
    # 2) Levantamos el modelo seleccionado desde nuestro servidor local
    if data['model'] == 'math':
        
        # Cargamos el modelos desde nuestra carpeta local en el servidor usando Pickle
        with open('./math_model.pkl', 'rb') as f_math:
            modelo_matematicas = pickle.load(f_math)
        
        m = modelo_matematicas
        
    else:
        return 'modelo %s no existe' % model
        
    predictions = m.predict([[int(d) for d in  data['dummies']]])
    
    return jsonify({'result': predictions[0]})

In [ ]:
# Ponemos a correr el servidor. Dado que ahora vamos a usar el método POST para pasar la información, ya no podemos
# hacerlo a partir de informació que esté en la URL. Así que para poder hacer un request con POST, mientras esta 
# notebook queda corriendo manteniendo nuestra API "online" (localmente), vamos a ir a la siguiente notebook: 
# "02b_Flask.ipynb" y hacer el request con POST: 
app.run(host='0.0.0.0', port = 5007)

<a href='#tabla_contenidos'>Volver a TOC</a>